In [ ]:
# Fetch signature url, then fetch content with wget:
# wget --wait=2 --random-wait --input-file=url.txt

In [6]:
def signaturePageUrl(index):
    return "https://www.mesopinions.com/petition/enfants/port-masque-6-ans/112026/page{}?signature-list=true".format(index)
    
pageIndex = 1
pageMax = 3959
hasSignature = True
urlList = []
    
while pageIndex < pageMax:
    urlList.append(signaturePageUrl(pageIndex))
    pageIndex += 1

In [9]:
%%capture cap --no-stderr
for url in urlList:
    print(url)

In [10]:
with open('no_mask/url.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
# Parse html content to get signature cards

In [1]:
from pyquery import PyQuery as pq
import glob
from datetime import datetime

rawCards = []
cards = []

file_list = glob.glob("no_mask/page*") # Include slash or it will search in the wrong directory!!
#print('file_list {}'.format(file_list))

for filename in file_list:
    d = pq(filename=filename)
    rawCards = rawCards + d(".mo-sign-card")
    
for card in rawCards:
    raw = d(card)
    cards.append({
        "id": raw(".f-bold").text().split(".")[0],
        "username": raw(".f-bold").text().split(".")[1].strip(),
        "timestamp":  datetime.strptime(raw(".sign-date").text(), 'Le %d/%m/%Y à %H:%M:%S'),
        "location": raw(".sign-ville").text(),
    })

print(cards[10])


{'id': '174480', 'username': 'Andrea', 'timestamp': datetime.datetime(2020, 11, 3, 22, 5, 13), 'location': ''}


In [2]:
import collections

locations = [c['location'].lower() for c in cards if len(c['location']) > 1]
counter = collections.Counter(locations)

print(counter)

Counter({'paris': 633, 'marseille': 371, 'nantes': 322, 'toulouse': 278, 'lyon': 241, 'nice': 233, 'montpellier': 181, 'strasbourg': 117, 'bordeaux': 116, 'rennes': 106, 'grenoble': 96, 'toulon': 96, 'lille': 86, 'villeurbanne': 83, 'angers': 78, 'aix en provence': 75, 'avignon': 73, 'perpignan': 63, 'tours': 57, 'metz': 56, 'annecy': 56, 'reims': 55, 'antibes': 52, 'nimes': 49, 'montreuil': 49, 'la rochelle': 47, 'brest': 47, 'dijon': 44, 'amiens': 43, 'cannes': 42, 'pau': 42, 'le mans': 42, 'nanterre': 41, 'argenteuil': 39, 'le havre': 39, 'courbevoie': 39, 'martigues': 38, 'menton': 38, 'tourcoing': 37, 'bruxelles': 37, 'albi': 36, 'la ciotat': 35, 'clermont ferrand': 35, 'ajaccio': 35, 'rouen': 34, 'besançon': 34, 'valence': 34, 'annemasse': 34, 'cagnes sur mer': 33, 'venissieux': 33, 'champigny sur marne': 32, 'clermont-ferrand': 31, 'versailles': 31, 'la seyne sur mer': 31, 'orleans': 31, 'limoges': 31, 'aubagne': 30, 'mulhouse': 30, 'nancy': 29, 'nîmes': 29, 'gennevilliers': 29,

In [25]:
import folium
from geopy.geocoders import Nominatim
import collections

m = folium.Map(
    location=[47, 2.21],
    tiles='Stamen Toner',
    zoom_start=6
)

mostCommon = 100
ratio = 100

locator = Nominatim(user_agent="myGeocoder")
locations = [c['location'].lower() for c in cards if len(c['location']) > 1]
c = collections.Counter(locations)

for loc, count in c.most_common(mostCommon):
    location = locator.geocode("{}, France".format(loc))

    folium.Circle(
        radius=count * ratio,
        location=[location.latitude, location.longitude],
        color='crimson',
        fill=False,
    ).add_to(m)

m